In [3]:
import keras
from keras import layers
import keras_tuner as kt
import pickle
from sklearn.model_selection import train_test_split

In [5]:
def model_builder(hp):
    model = keras.Sequential()
    
    # First Conv Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_1_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Second Conv Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_2_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
        activation='relu'
    ))
    model.add(layers.MaxPooling2D(pool_size=(2, 2)))

    # Third Conv Layer
    model.add(layers.Conv2D(
        filters=hp.Int('conv_3_filters', min_value=32, max_value=128, step=32),
        kernel_size=hp.Choice('conv_3_kernel', values=[3, 5]),
        activation='relu'
    ))

    model.add(layers.Flatten())

    # Fully Connected Dense Layer
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=32, max_value=128, step=32),
        activation='relu'
    ))

    # Output Layer
    model.add(layers.Dense(6, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [4]:
# Load already augmented data from saved pickle files
X_train_aug = pickle.load(open("../../pickles/X_train_aug.pkl", "rb"))
y_train_aug = pickle.load(open("../../pickles/y_train_aug.pkl", "rb"))
X_test = pickle.load(open("../../pickles/X_test.pkl", "rb"))
y_test = pickle.load(open("../../pickles/y_test.pkl", "rb"))

# further split the training data into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_aug, y_train_aug, test_size=0.2, random_state=42)

In [6]:
# Initialize the tuner
tuner = kt.RandomSearch(
    model_builder,
    objective='val_accuracy',
    max_trials=10,  # Number of different models to try
    executions_per_trial=1,  # Number of times to train each model
    directory='keras_tuner_results',
    project_name='cnn_tuning'
)

# Perform Hyperparameter Search
tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Get the Best Hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build and Train the Best Model
best_model = tuner.hypermodel.build(best_hps)
history = best_model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Evaluate on Test Data
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

Trial 10 Complete [00h 45m 34s]
val_accuracy: 0.8128783702850342

Best val_accuracy So Far: 0.8130618333816528
Total elapsed time: 09h 10m 56s
Epoch 1/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 272s 398ms/step - accuracy: 0.5659 - loss: 1.1200 - val_accuracy: 0.7118 - val_loss: 0.7844
Epoch 2/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 259s 379ms/step - accuracy: 0.7345 - loss: 0.7075 - val_accuracy: 0.7311 - val_loss: 0.7374
Epoch 3/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 258s 378ms/step - accuracy: 0.7956 - loss: 0.5591 - val_accuracy: 0.7736 - val_loss: 0.6216
Epoch 4/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 259s 380ms/step - accuracy: 0.8340 - loss: 0.4580 - val_accuracy: 0.7850 - val_loss: 0.6010
Epoch 5/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 257s 377ms/step - accuracy: 0.8628 - loss: 0.3821 - val_accuracy: 0.7894 - val_loss: 0.6089
Epoch 6/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 257s 377ms/step - accuracy: 0.8949 - loss: 0.3037 - val_accuracy: 0.7921 - val_loss: 0.6056
Epoch 7/20
682/682 ━━━━━━━━━━━━━━━━━━━━ 258s 378ms/step - accurac

In [8]:
display(best_hps.values)

{'conv_1_filters': 32,
 'conv_1_kernel': 5,
 'conv_2_filters': 64,
 'conv_2_kernel': 5,
 'conv_3_filters': 128,
 'conv_3_kernel': 3,
 'dense_units': 128,
 'learning_rate': 0.0001}

In [7]:
best_model.save("../../models/tuned_model.keras")